<a href="https://colab.research.google.com/github/ImperB-riven/company/blob/main/cbp_standard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_fscore_support as score
from xgboost import XGBClassifier

from IPython.display import display, Image

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# !gdown --id 1czQ37hv9D1h6Un27sjeJmJlRGi2rPwWh
# # https://drive.google.com/file/d/1czQ37hv9D1h6Un27sjeJmJlRGi2rPwWh/view?usp=sharing

# !gdown --id 1pBMPxT0FoCxDsF4h3ryka_7A_pA47O2P
# # https://drive.google.com/file/d/1pBMPxT0FoCxDsF4h3ryka_7A_pA47O2P/view?usp=sharing

# !gdown --id 1OnmOSgjRyqe81BiJNA7rWc9O4CVHfT28
# # https://drive.google.com/file/d/1OnmOSgjRyqe81BiJNA7rWc9O4CVHfT28/view?usp=sharing



In [4]:
def find_missing_percent(data, showresult = True):
  miss_df = pd.DataFrame ({'ColumnName':[], 'TotalMissingVals':[], 'PercentMissing':[]})
  for col in data.columns:
    sum_miss_val = data[col].isnull () .sum( )
    percent_miss_val = round ((sum_miss_val / data.shape[0]) * 100,2)
    missinginfo = {"ColumnName" : col, "TotalMissingVals" : sum_miss_val, "PercentMissing" : percent_miss_val}
    miss_df = miss_df. append(missinginfo, ignore_index = True)
  miss_df = miss_df [miss_df ["PercentMissing"] >0.0]
  miss_df = miss_df. reset_index(drop = True)
  miss_features = miss_df [ "ColumnName"].values
  if (showresult):
    print(data. shape)
    display (data. head())
    display (miss_df)
  return miss_df

In [5]:
df = pd.read_csv('/content/gdrive/MyDrive/cbp/train.csv')
df = df.drop(columns='id')
df.head().style

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,4,6,7,8,10,11,12,15,16,19,23,26,30,36,37,39,49,52,56,58,59,60) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,forecasting period,net_profit_over_total_assets,total_liabilities_over_total_assets,working_capital_over_total_assets,current_assets_over_short_term_liabilities,cash_plus_short_term_securities_plus_receivables_minus_short_term_liabilities_over_operating_expenses_minus_depreciation_times_365,retained_earnings_over_total_assets,EBIT_over_total_assets,book_value_of_equity_over_total_liabilities,sales_over_total_assets,equity_over_total_assets,gross_profit_plus_extraordinary_items_plus_financial_expenses_over_total_assets,gross_profit_over_short_term_liabilities,gross_profit_plus_depreciation_over_sales,gross_profit_plus_interest_over_total_assets,total_liabilities_times_365_over_gross_profit_plus_depreciation,gross_profit_plus_depreciation_over_total_liabilities,total_assets_over_total_liabilities,gross_profit_over_total_assets,gross_profit_over_sales,inventory_times_365_over_sales,sales_n_over_sales_n_minus_1,profit_on_operating_activities_over_total_assets,net_profit_over_sales,gross_profit_in_3_years_over_total_assets,equity_minus_share_capital_over_total_assets,net_profit_plus_depreciation_over_total_liabilities,profit_on_operating_activities_over_financial_expenses,working_capital_over_fixed_assets,logarithm_of_total_assets,total_liabilities_minus_cash_over_sales,gross_profit_plus_interest_over_sales,current_liabilities_times_365_over_cost_of_products_sold,operating_expenses_over_short_term_liabilities,operating_expenses_over_total_liabilities,profit_on_sales_over_total_assets,total_sales_over_total_assets,current_assets_minus_inventories_over_long_term_liabilities,constant_capital_over_total_assets,profit_on_sales_over_sales,current_assets_minus_inventory_minus_receivables_over_short_term_liabilities,total_liabilities_over_profit_on_operating_activities_plus_depreciation_times_12_over_365,profit_on_operating_activities_over_sales,rotation_receivables_plus_inventory_turnover_in_days,receivables_times_365_over_sales,net_profit_over_inventory,current_assets_minus_inventory_over_short_term_liabilities,inventory_times_365_over_cost_of_products_sold,EBITDA_profit_on_operating_activities_minus_depreciation_over_total_assets,EBITDA_profit_on_operating_activities_minus_depreciation_over_sales,current_assets_over_total_liabilities,short_term_liabilities_over_total_assets,short_term_liabilities_times_365_over_cost_of_products_sold,equity_over_fixed_assets,constant_capital_over_fixed_assets,working_capital,sales_minus_cost_of_products_sold_over_sales,current_assets_minus_inventory_minus_short_term_liabilities_over_sales_minus_gross_profit_minus_depreciation,total_costs_overtotal_sales,long_term_liabilities_over_equity,sales_over_inventory,sales_over_receivables,short_term_liabilities_times_365_over_sales,sales_over_short_term_liabilities,sales_over_fixed_assets,class
0,4,0.141939,0.450292,0.021422,1.0529865561090996,-28.529083790353447,0.141934,0.172629,0.9903370061983934,1.0873559014030911,0.445948,0.17263990089441028,0.4285853456426442,0.09848202407105192,0.172624,737.5983611071007,0.49502168637120986,2.220461536498334,0.172637,0.07619762337252026,28.299284841391472,1.0284250537325912,0.189883,0.06267452756868666,0.17262730881422864,0.445979,0.426728804736232,?,0.03708253810878011,4.572500,0.19211349076145456,0.07614026239779019,70.55540862591312,5.173970681199211,0.4217447508375348,0.189890,2.290576,5.224748457992206,0.493565,0.0837302731954118,0.05748447120951893,0.059289848754031045,0.08376464794297411,64.96041100046314,36.353133014029936,0.8090845424086093,0.6173668337183532,30.730599182210614,0.13965781638150723,0.061639279615216674,0.9416512291107305,0.402678,0.19195545568684175,0.7759224890090642,0.8564858473735182,800.542408,0.07957187794085908,0.318286,0.9276869666991867,0.106818,12.941797286204972,10.044584894139062,64.71369972433212,5.6268942646021545,3.9370641007164138,1
1,4,0.147560,0.234840,0.708335,4.016018367133703,119.44561073601812,0.457090,0.186070,3.2064449649495983,1.0934040907472207,0.753147,0.18607412665007764

In [6]:
df

,forecasting period,net_profit_over_total_assets,total_liabilities_over_total_assets,working_capital_over_total_assets,current_assets_over_short_term_liabilities,cash_plus_short_term_securities_plus_receivables_minus_short_term_liabilities_over_operating_expenses_minus_depreciation_times_365,retained_earnings_over_total_assets,EBIT_over_total_assets,book_value_of_equity_over_total_liabilities,sales_over_total_assets,...,sales_minus_cost_of_products_sold_over_sales,current_assets_minus_inventory_minus_short_term_liabilities_over_sales_minus_gross_profit_minus_depreciation,total_costs_overtotal_sales,long_term_liabilities_over_equity,sales_over_inventory,sales_over_receivables,short_term_liabilities_times_365_over_sales,sales_over_short_term_liabilities,sales_over_fixed_assets,class
0,4,0.141939,0.450292,0.021422,1.0529865561090996,-28.529083790353447,0.141934,0.172629,0.9903370061983934,1.0873559014030911,...,0.07957187794085908,0.318286,0.9276869666991867,0.106818,12.941797286204972,10.044584894139062,64.71369972433212,5.6268942646021545,3.9370641007164138,1
1,4,0.14756,0.23484,0.708335,4.016018367133703,119.44561073601812,0.45709,0.18607,3.2064449649495983,1.0934040907472207,...,0.09521770610492689,0.195943,0.9070561344356298,-0.000029,233.30168832258815,2.7474158180607167,36.967434520776315,9.813419478357071,40.56773917968368,0
2,1,-0.024542,0.311544,0.067831,1.2552471026972796,-15.01992268530473,-0.657003,-0.024537,2.209000357935884,0.8255784100560334,...,0.10903452563942133,-0.035634,0.9298251966432225,0.055331,8.589990830766858,5.164737918343701,117.92403966526238,3.098272636244345,1.2397105028735345,0
3,2,0.159317,0.777017,0.207431,1.2673100488694038,-4.892917756326703,-0.000005,0.159321,0.2881773632418629,2.2835319735778707,...,0.10297114004320836,0.714528,0.9378954347737054,-0.000077,9.92822118614928,5.91278624799987,123.7526101491789,2.9460083818399756,130.2083212255033,0
4,4,0.004404,0.577419,-0.104435,0.7862247169819696,-116.9116175491663,0.056561,0.004249,0.6921661681503327,1.013604581322834,...,0.016446056830358928,0.011019,0.9794252272096882,0.221335,5.099540696974632,6.890130245899504,166.7619795766402,2.189705498789674,1.7403533821469037,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25116,4,0.24623,0.170407,0.813493,5.773889138790976,213.048188,-0.000009,0.302477,4.868535164929246,1.301126,...,0.2283769027851694,0.29681668818505214,0.763231,7.70633975208323e-05,5.897284079340597,3.7231830654998537,47.73240545990899,7.635120181636164,80.72888742700145,0
25117,3,-0.059211,0.721817,-0.258946,0.5647361245781731,-138.046387,-0.333938,-0.050821,0.38630606957673463,1.251287,...,-0.04056480815026849,-0.21287829040021203,1.019349,0.36809270796442145,5.266247654897824,16.21847260371964,173.8629453484833,2.1018924419657847,1.8863095038196047,1
25118,1,0.045154,0.374294,-0.100998,0.4792078614736359,-85.886817,0.12329,0.056916,1.5239007689822572,1.071334,...,0.06822921550523516,0.07916795520304219,0.931599,0.3161383818368783,15.40052287960648,20.053088839146028,98.83640557122028,3.6926972667134454,0.7913804872914476,0
25119,1,-0.022056,0.877403,-0.121055,0.86133420846161,-122.361688,-0.042067,-0.019704,0.10910412587420266,1.003037,...,8.69314128017322e-05,-0.23053357833553179,0.997731,0.033051271974198236,4.3450356846014,2.712144604667035,260.217704529575,1.403049813033113,4.968499800731599,0


In [7]:
df= df.replace('?', np.nan)

for i in df.columns[1:-1]:
    df[i]=df[i].astype(float)

In [8]:
miss_df = find_missing_percent(df)
drop_cols = list(miss_df[miss_df['PercentMissing'] > 20.0].ColumnName)
drop_cols

(25121, 66)


,forecasting period,net_profit_over_total_assets,total_liabilities_over_total_assets,working_capital_over_total_assets,current_assets_over_short_term_liabilities,cash_plus_short_term_securities_plus_receivables_minus_short_term_liabilities_over_operating_expenses_minus_depreciation_times_365,retained_earnings_over_total_assets,EBIT_over_total_assets,book_value_of_equity_over_total_liabilities,sales_over_total_assets,...,sales_minus_cost_of_products_sold_over_sales,current_assets_minus_inventory_minus_short_term_liabilities_over_sales_minus_gross_profit_minus_depreciation,total_costs_overtotal_sales,long_term_liabilities_over_equity,sales_over_inventory,sales_over_receivables,short_term_liabilities_times_365_over_sales,sales_over_short_term_liabilities,sales_over_fixed_assets,class
0,4,0.141939,0.450292,0.021422,1.052987,-28.529084,0.141934,0.172629,0.990337,1.087356,...,0.079572,0.318286,0.927687,0.106818,12.941797,10.044585,64.713700,5.626894,3.937064,1
1,4,0.147560,0.234840,0.708335,4.016018,119.445611,0.457090,0.186070,3.206445,1.093404,...,0.095218,0.195943,0.907056,-0.000029,233.301688,2.747416,36.967435,9.813419,40.567739,0
2,1,-0.024542,0.311544,0.067831,1.255247,-15.019923,-0.657003,-0.024537,2.209000,0.825578,...,0.109035,-0.035634,0.929825,0.055331,8.589991,5.164738,117.924040,3.098273,1.239711,0
3,2,0.159317,0.777017,0.207431,1.267310,-4.892918,-0.000005,0.159321,0.288177,2.283532,...,0.102971,0.714528,0.937895,-0.000077,9.928221,5.912786,123.752610,2.946008,130.208321,0
4,4,0.004404,0.577419,-0.104435,0.786225,-116.911618,0.056561,0.004249,0.692166,1.013605,...,0.016446,0.011019,0.979425,0.221335,5.099541,6.890130,166.761980,2.189705,1.740353,0


,ColumnName,TotalMissingVals,PercentMissing
0,net_profit_over_total_assets,3.0,0.01
1,total_liabilities_over_total_assets,3.0,0.01
2,working_capital_over_total_assets,3.0,0.01
3,current_assets_over_short_term_liabilities,78.0,0.31
4,cash_plus_short_term_securities_plus_receivabl...,57.0,0.23
...,...,...,...
58,sales_over_inventory,1282.0,5.10
59,sales_over_receivables,57.0,0.23
60,short_term_liabilities_times_365_over_sales,77.0,0.31
61,sales_over_short_term_liabilities,78.0,0.31


['current_assets_minus_inventories_over_long_term_liabilities']

In [9]:
def mean_imputation(data_numeric):
    for col in data_numeric.columns:
        mean = data_numeric[col].mean()
        data_numeric[col] = data_numeric[col].fillna(mean)
    return data_numeric

In [10]:
df = df.astype(float)

In [11]:
df = mean_imputation(df)

In [12]:
find_missing_percent(df)

(25121, 66)


,forecasting period,net_profit_over_total_assets,total_liabilities_over_total_assets,working_capital_over_total_assets,current_assets_over_short_term_liabilities,cash_plus_short_term_securities_plus_receivables_minus_short_term_liabilities_over_operating_expenses_minus_depreciation_times_365,retained_earnings_over_total_assets,EBIT_over_total_assets,book_value_of_equity_over_total_liabilities,sales_over_total_assets,...,sales_minus_cost_of_products_sold_over_sales,current_assets_minus_inventory_minus_short_term_liabilities_over_sales_minus_gross_profit_minus_depreciation,total_costs_overtotal_sales,long_term_liabilities_over_equity,sales_over_inventory,sales_over_receivables,short_term_liabilities_times_365_over_sales,sales_over_short_term_liabilities,sales_over_fixed_assets,class
0,4.0,0.141939,0.450292,0.021422,1.052987,-28.529084,0.141934,0.172629,0.990337,1.087356,...,0.079572,0.318286,0.927687,0.106818,12.941797,10.044585,64.713700,5.626894,3.937064,1.0
1,4.0,0.147560,0.234840,0.708335,4.016018,119.445611,0.457090,0.186070,3.206445,1.093404,...,0.095218,0.195943,0.907056,-0.000029,233.301688,2.747416,36.967435,9.813419,40.567739,0.0
2,1.0,-0.024542,0.311544,0.067831,1.255247,-15.019923,-0.657003,-0.024537,2.209000,0.825578,...,0.109035,-0.035634,0.929825,0.055331,8.589991,5.164738,117.924040,3.098273,1.239711,0.0
3,2.0,0.159317,0.777017,0.207431,1.267310,-4.892918,-0.000005,0.159321,0.288177,2.283532,...,0.102971,0.714528,0.937895,-0.000077,9.928221,5.912786,123.752610,2.946008,130.208321,0.0
4,4.0,0.004404,0.577419,-0.104435,0.786225,-116.911618,0.056561,0.004249,0.692166,1.013605,...,0.016446,0.011019,0.979425,0.221335,5.099541,6.890130,166.761980,2.189705,1.740353,0.0


,ColumnName,TotalMissingVals,PercentMissing


,ColumnName,TotalMissingVals,PercentMissing


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [14]:
X=df.drop(columns='class')
y=df['class']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [16]:
# model = RandomForestClassifier(random_state=1)
# model.fit(X_train, y_train)
# # make predictions
# yhat = model.predict(X_test)
# # evaluate predictions
# acc = accuracy_score(y_test, yhat)
# print('Accuracy: %.3f' % acc)

# test

In [17]:
df_test = pd.read_csv('/content/gdrive/MyDrive/cbp/test.csv')
df_test = df_test.drop(columns='id')

df_test

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,4,7,8,10,11,15,19,23,26,30,36,37,39,49,52,58,60) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,forecasting period,net_profit_over_total_assets,total_liabilities_over_total_assets,working_capital_over_total_assets,current_assets_over_short_term_liabilities,cash_plus_short_term_securities_plus_receivables_minus_short_term_liabilities_over_operating_expenses_minus_depreciation_times_365,retained_earnings_over_total_assets,EBIT_over_total_assets,book_value_of_equity_over_total_liabilities,sales_over_total_assets,...,working_capital,sales_minus_cost_of_products_sold_over_sales,current_assets_minus_inventory_minus_short_term_liabilities_over_sales_minus_gross_profit_minus_depreciation,total_costs_overtotal_sales,long_term_liabilities_over_equity,sales_over_inventory,sales_over_receivables,short_term_liabilities_times_365_over_sales,sales_over_short_term_liabilities,sales_over_fixed_assets
0,4,0.087054,0.504697,-0.213442,0.5272011313535183,-945.320442201966,0.0,0.091917,0.9822931260560077,1.642851,...,-19987.124701,0.32238638629656047,0.175775,0.706656279337095,0.010392,28.87264617446681,12.709310879310008,100.29264324183652,3.644498341924968,2.154215441909059
1,2,-0.21039,0.631828,0.228799,1.3699504944224292,-63.35127273935373,-0.038446,-0.258109,0.5828861702085418,1.670711,...,18.621606,-0.12673588532242966,-0.571445,1.1529156062660915,0.00004,3.070083299679344,6.488414971863619,135.02740903472719,2.7005476588652746,10.951484197838264
2,1,0.755349,0.208157,0.525567,3.525034130683577,95.93687042196258,0.007305,0.755342,3.804564443625834,2.604779,...,1440.011406,0.33790489645280797,0.953913,0.7087001780697151,0.000308,34.48170205384968,5.197885219493402,29.17870298751135,12.513041883618936,9.78210680456828
3,2,0.090151,0.539555,0.220522,1.409650590861931,-8.775939570005415,-0.316524,0.090153,0.8526744012682892,2.567658,...,503.854614,0.22067902616244436,0.195811,0.7763413442483931,0.000179,10.178220145524635,5.473610577486383,76.57595557247394,4.771107985965157,10.639494588861265
4,4,0.023989,0.002572,0.987142,385.5699438551996,1704.2279872516292,0.000014,0.02951,388.58956400997704,0.210381,...,12841.024909,0.05181620680909046,0.024058,0.8841554810647239,0.000004,5.369218427187152,14.109096625044295,4.34000451937428,81.96714896644555,20.421013176211684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12369,4,0.01855753489301011,0.41243481190545667,0.4885099046644078,2.184302748235379,-14.33278910047721,-1.368740926711409e-06,0.02649786330208904,1.4254554560441222,1.9548699171816462,...,7173.998704,0.028747833336799446,0.03157725064814445,0.9839134495540388,-2.6846638023783474e-05,3.6336700206487773,5.911055062151485,76.93098422978343,4.739695631860874,19.736004930040917
12370,3,0.19222944212914414,0.524764034767894,0.17805143973397178,1.7276822729406525,23.932394895256063,2.1075129188998702e-06,0.2296875432559015,0.9074577581205645,1.737568655857094,...,1269.036044,0.2624115211624015,0.4045052488116174,0.7456741549963615,0.0002232371804427739,24.942178342696415,29.24329786963163,51.25703346044781,7.100221267087136,3.00851786947125
12371,1,0.12697056258115008,0.3723415844371529,0.0985406295212339,1.3387023014055945,-16.91990047213155,-2.660530677949785e-06,0.15906798420044346,1.6858074386662574,1.9154270706135945,...,989.486683,0.09593470408413801,0.20229441879485618,0.9189662476296417,0.12989152337270907,11.00661478925832,9.223364475947358,55.342409011295196,6.58785040563515,3.140681471190523
12372,3,0.025289897142166087,0.5324650059180629,0.07265672486362844,1.1546477275967073,-29.79489078104177,0.10828175782584397,0.031503697158378016,0.8777572186366555,0.862947368579279,...,22251.005795,0.18710145634429662,0.05408808514717573,0.8445094770104019,0.13546840828282325,6.995182185572398,2.097491682047232,198.184874787785,1.8400022423349767,1.8874250395996603


In [18]:
df_test = df_test.replace('?', np.nan)

df_test = df_test.astype(float)

In [19]:
miss_df_test = find_missing_percent(df_test)
drop_cols_test = list(miss_df_test[miss_df_test['PercentMissing'] > 20.0].ColumnName)
drop_cols_test

(12374, 65)


,forecasting period,net_profit_over_total_assets,total_liabilities_over_total_assets,working_capital_over_total_assets,current_assets_over_short_term_liabilities,cash_plus_short_term_securities_plus_receivables_minus_short_term_liabilities_over_operating_expenses_minus_depreciation_times_365,retained_earnings_over_total_assets,EBIT_over_total_assets,book_value_of_equity_over_total_liabilities,sales_over_total_assets,...,working_capital,sales_minus_cost_of_products_sold_over_sales,current_assets_minus_inventory_minus_short_term_liabilities_over_sales_minus_gross_profit_minus_depreciation,total_costs_overtotal_sales,long_term_liabilities_over_equity,sales_over_inventory,sales_over_receivables,short_term_liabilities_times_365_over_sales,sales_over_short_term_liabilities,sales_over_fixed_assets
0,4.0,0.087054,0.504697,-0.213442,0.527201,-945.320442,2.456644e-07,0.091917,0.982293,1.642851,...,-19987.124701,0.322386,0.175775,0.706656,0.010392,28.872646,12.709311,100.292643,3.644498,2.154215
1,2.0,-0.210390,0.631828,0.228799,1.369950,-63.351273,-3.844551e-02,-0.258109,0.582886,1.670711,...,18.621606,-0.126736,-0.571445,1.152916,0.000040,3.070083,6.488415,135.027409,2.700548,10.951484
2,1.0,0.755349,0.208157,0.525567,3.525034,95.936870,7.305173e-03,0.755342,3.804564,2.604779,...,1440.011406,0.337905,0.953913,0.708700,0.000308,34.481702,5.197885,29.178703,12.513042,9.782107
3,2.0,0.090151,0.539555,0.220522,1.409651,-8.775940,-3.165242e-01,0.090153,0.852674,2.567658,...,503.854614,0.220679,0.195811,0.776341,0.000179,10.178220,5.473611,76.575956,4.771108,10.639495
4,4.0,0.023989,0.002572,0.987142,385.569944,1704.227987,1.438131e-05,0.029510,388.589564,0.210381,...,12841.024909,0.051816,0.024058,0.884155,0.000004,5.369218,14.109097,4.340005,81.967149,20.421013


,ColumnName,TotalMissingVals,PercentMissing
0,net_profit_over_total_assets,2.0,0.02
1,total_liabilities_over_total_assets,2.0,0.02
2,working_capital_over_total_assets,2.0,0.02
3,current_assets_over_short_term_liabilities,35.0,0.28
4,cash_plus_short_term_securities_plus_receivabl...,21.0,0.17
...,...,...,...
58,sales_over_inventory,602.0,4.87
59,sales_over_receivables,30.0,0.24
60,short_term_liabilities_times_365_over_sales,50.0,0.40
61,sales_over_short_term_liabilities,35.0,0.28


['current_assets_minus_inventories_over_long_term_liabilities']

In [20]:
df_test = mean_imputation(df_test)

In [21]:
find_missing_percent(df_test)

(12374, 65)


,forecasting period,net_profit_over_total_assets,total_liabilities_over_total_assets,working_capital_over_total_assets,current_assets_over_short_term_liabilities,cash_plus_short_term_securities_plus_receivables_minus_short_term_liabilities_over_operating_expenses_minus_depreciation_times_365,retained_earnings_over_total_assets,EBIT_over_total_assets,book_value_of_equity_over_total_liabilities,sales_over_total_assets,...,working_capital,sales_minus_cost_of_products_sold_over_sales,current_assets_minus_inventory_minus_short_term_liabilities_over_sales_minus_gross_profit_minus_depreciation,total_costs_overtotal_sales,long_term_liabilities_over_equity,sales_over_inventory,sales_over_receivables,short_term_liabilities_times_365_over_sales,sales_over_short_term_liabilities,sales_over_fixed_assets
0,4.0,0.087054,0.504697,-0.213442,0.527201,-945.320442,2.456644e-07,0.091917,0.982293,1.642851,...,-19987.124701,0.322386,0.175775,0.706656,0.010392,28.872646,12.709311,100.292643,3.644498,2.154215
1,2.0,-0.210390,0.631828,0.228799,1.369950,-63.351273,-3.844551e-02,-0.258109,0.582886,1.670711,...,18.621606,-0.126736,-0.571445,1.152916,0.000040,3.070083,6.488415,135.027409,2.700548,10.951484
2,1.0,0.755349,0.208157,0.525567,3.525034,95.936870,7.305173e-03,0.755342,3.804564,2.604779,...,1440.011406,0.337905,0.953913,0.708700,0.000308,34.481702,5.197885,29.178703,12.513042,9.782107
3,2.0,0.090151,0.539555,0.220522,1.409651,-8.775940,-3.165242e-01,0.090153,0.852674,2.567658,...,503.854614,0.220679,0.195811,0.776341,0.000179,10.178220,5.473611,76.575956,4.771108,10.639495
4,4.0,0.023989,0.002572,0.987142,385.569944,1704.227987,1.438131e-05,0.029510,388.589564,0.210381,...,12841.024909,0.051816,0.024058,0.884155,0.000004,5.369218,14.109097,4.340005,81.967149,20.421013


,ColumnName,TotalMissingVals,PercentMissing


,ColumnName,TotalMissingVals,PercentMissing


In [22]:
x_test2 = df_test

# y_pred2 = model.predict(x_test2)

In [23]:
y_test2 = pd.read_csv('/content/gdrive/MyDrive/cbp/sample_submission.csv')

y_test2 = y_test2.drop(columns='id')
y_test2 = y_test2.astype(float)

In [24]:
# acc2 = accuracy_score(y_test2, y_pred2)
# print('Accuracy: %.3f' % acc2)

In [25]:
# def RandomForest():
#   for i in range(1,4):
#     model = RandomForestClassifier(random_state=i)
#     model.fit(X_train, y_train)
#     # make predictions
#     yhat = model.predict(X_test)
#     # evaluate predictions
#     acc = accuracy_score(y_test, yhat)
#     print('Accuracy: %.3f' % acc)
#     y_pred2 = model.predict(x_test2)
#     acc2 = accuracy_score(y_test2, y_pred2)
#     print('Accuracy: %.3f' % acc2)  

In [26]:
# RandomForest()

In [27]:
# sub = pd.DataFrame(y_pred2)

In [28]:
# sub.head()

In [30]:
# sub.index = sub.index+25121

In [31]:
# sub['class'] = sub[0]

# sub= sub.drop(columns=0)

# sub

In [32]:
# sub.to_csv('sub0505.csv')